In [1]:
#importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#the years we wish to parse for
years = list(range(1990, 2017))

#list to house dataframes
dfs = []

#list to house collection of player IDs
player_ids = []

#unnecessary table headers to be removed
classes = ["over_header","over_header thead", "thead"]

In [4]:
#   "main" for loop to iterate through HTML pages
for year in years:
### Downloading HTML block ### 
#   #variables serve to gather running site pages
#   url_base = "https://www.basketball-reference.com/leagues/NBA_{}_rookies-season-stats.html"
#   url = url_base.format(year)
#   data = requests.get(url)
#   
#   #page is saved as an html and placed in Rookies folder
#   with open("Rookies/{}.html".format(year), "w+") as f:
#       f.write(data.text)
    
#   open and read iterated year's HTML page    
    with open("Pages/{}.html".format(year), encoding = "utf-8") as f:
        page = f.read()
        
#   running page through Beautiful Soup creates BeautifulSoup object
    soup = BeautifulSoup(page, "html.parser")
    
#   for loop cleans up unnecessary table headers from reappearing in rows
    for i in classes:
        for tr in soup.find_all("tr", {"class":i}):
            tr.decompose()
    
### Scraping Block ###
#   identifies, scrapes, and loads rookie table into one dataframe
    rookie_table = soup.find(id="rookies")
    rookies = pd.read_html(str(rookie_table))[0]

#   "Year" column created and cells filled with corresponding interated rookie class year    
    rookies["Year"] = year

#   Basketball-Reference player ID extracted and added to global list variable "player_ids"
    for tr in soup.select("tr.full_table"):
        for td in tr.select("td")[0:1]:
            player_ids.append(td['data-append-csv'])

#   iterated year's dataframe added to global list variable "dfs"     
    dfs.append(rookies)

#   close file
    f.close()
    
#end for loop through years

In [5]:
player_ids.dtype

AttributeError: 'list' object has no attribute 'dtype'

In [6]:
#turns list of dataframes into single dataframe
all_rookies = pd.concat(dfs)

In [ ]:
#drop unncessary column 'Rk' from dataframe
all_rookies = all_rookies.drop(columns='Rk')

In [7]:
#create column in dataframe, fill with Basketball-Reference player id list
all_rookies["BR ID"] = player_ids

In [10]:
all_rookies.dtypes

Rk          int64
Player     object
Debut      object
Age         int64
Yrs         int64
G           int64
MP          int64
FG          int64
FGA         int64
3P          int64
3PA         int64
FT          int64
FTA         int64
ORB         int64
TRB         int64
AST         int64
STL         int64
BLK         int64
TOV         int64
PF          int64
PTS         int64
FG%       float64
3P%       float64
FT%       float64
MP.1      float64
PTS.1     float64
TRB.1     float64
AST.1     float64
Year        int64
BR ID      object
dtype: object

In [ ]:
#dataframe archived as local CSV sans index
all_rookies.to_csv("Spreadsheets/NBA_Rookies_90-16.csv", index=False)